In [36]:
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
import string
from nltk.corpus import stopwords
import pandas as pd
from nltk.stem import WordNetLemmatizer

import numpy as np
import re
import nltk
import spacy
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
# Load the CSV file
df = pd.read_csv('/content/Tag_dataset_tags.csv')

# Data Cleaning
df.dropna(subset=['Description'], inplace=True)  # Drop rows with missing descriptions
df.drop_duplicates(subset=['Description'], keep='first', inplace=True)  # Remove duplicates

# Text Preprocessing with Lemmatization and Stopword Removal
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    tokens = word_tokenize(text)
    lemmatized_tokens = [lemmatizer.lemmatize(word) for word in tokens]

    return ' '.join(lemmatized_tokens)

df['lemmatized_description'] = df['Description'].apply(lemmatize_text)

# Remove Stopwords from Lemmatized Text
def remove_stopwords(text):
    tokens = word_tokenize(text)
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]

    return ' '.join(filtered_tokens)

df['cleaned_lemmatized_description'] = df['lemmatized_description'].apply(remove_stopwords)

# Create Array of Words from Cleaned Lemmatized Description
def create_word_array(text):
    return word_tokenize(text)

df['word_array'] = df['cleaned_lemmatized_description'].apply(create_word_array)

# Perform POS Tagging on Each Word in Each Array
def pos_tag_words(word_array):
    return pos_tag(word_array)

df['pos_tagged_words'] = df['word_array'].apply(pos_tag_words)

# Create Separate Columns for Specific POS Tags
def extract_pos_tags(tagged_words, pos_tag):
    return ' '.join([word for word, tag in tagged_words if tag == pos_tag])

df['Nouns'] = df['pos_tagged_words'].apply(lambda x: extract_pos_tags(x, 'NN') + ' ' + extract_pos_tags(x, 'NNS') + ' ' + extract_pos_tags(x, 'NNP') + ' ' + extract_pos_tags(x, 'NNPS'))
df['Verbs'] = df['pos_tagged_words'].apply(lambda x: extract_pos_tags(x, 'VB') + ' ' + extract_pos_tags(x, 'VBD') + ' ' + extract_pos_tags(x, 'VBG') + ' ' + extract_pos_tags(x, 'VBN') + ' ' + extract_pos_tags(x, 'VBP') + ' ' + extract_pos_tags(x, 'VBZ'))
df['Adjectives'] = df['pos_tagged_words'].apply(lambda x: extract_pos_tags(x, 'JJ'))
# Save the preprocessed data with specific POS tagged words to a new CSV file
df.to_csv('Final_pd.csv', index=False)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [37]:
import tensorflow as tf
from tensorflow import keras

In [38]:
df.head()

,ID,Name,Description,Tags,lemmatized_description,cleaned_lemmatized_description,word_array,pos_tagged_words,Nouns,Verbs,Adjectives
0,1.0,Sarah,Sarah is a dedicated student with a passion fo...,Physics,Sarah is a dedicated student with a passion fo...,Sarah dedicated student passion Mathematics Ph...,"[Sarah, dedicated, student, passion, Mathemati...","[(Sarah, NNP), (dedicated, VBD), (student, NN)...",student passion interest lie realm fervor curi...,dedicated mystery,universe
1,2.0,Alex,Alex is a tech enthusiast studying Computer Sc...,Computer Science,Alex is a tech enthusiast studying Computer Sc...,Alex tech enthusiast studying Computer Science...,"[Alex, tech, enthusiast, studying, Computer, S...","[(Alex, NNP), (tech, VBD), (enthusiast, RB), (...",interest revolve technology computing Alex Co...,tech studying shaping explores,cutting-edge future
2,3.0,Emily,"Emily is an avid reader and historian, focusin...",Literature,"Emily is an avid reader and historian , focusi...","Emily avid reader historian , focusing Literat...","[Emily, avid, reader, historian, ,, focusing, ...","[(Emily, RB), (avid, JJ), (reader, NN), (histo...",reader historian inspiration endeavor Literat...,find focusing drawing,avid joy deep narrative creative
3,4.0,James,James is a science enthusiast excelling in Bio...,Chemistry,James is a science enthusiast excelling in Bio...,James science enthusiast excelling Biology Che...,"[James, science, enthusiast, excelling, Biolog...","[(James, NNP), (science, NN), (enthusiast, NN)...",science enthusiast interest research experimen...,excelling drive seek,unravel molecular
4,5.0,Mia,Mia is a student of Economics and Political Sc...,Economics,Mia is a student of Economics and Political Sc...,"Mia student Economics Political Science , pass...","[Mia, student, Economics, Political, Science, ...","[(Mia, NNP), (student, NN), (Economics, NNP), ...",student passionate understanding interest comm...,making informed reflect,global dynamic positive


In [60]:
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk import pos_tag

# Read the CSV file into a pandas DataFrame
file_path = "/content/Final_pd.csv"
data = pd.read_csv(file_path)

# Function to extract tags from description
def extract_tags(description):
    # Tokenize the description
    tokens = word_tokenize(description)
    # Perform Part-of-Speech tagging
    tagged_words = pos_tag(tokens)
    # Define your logic to extract tags, for simplicity, let's assume the first noun encountered is the tag
    tags = [word for word, tag in tagged_words if tag.startswith('N')]
    return tags

# Apply the function to each row to generate tags
data['Tags'] = data['Description'].apply(extract_tags)

# Save the updated DataFrame to a new CSV file if needed
output_file_path = "/content/Final_pd_with_tags.csv"
data.to_csv(output_file_path, index=False)

# Display the updated DataFrame
print(data)


      ID       Name                                        Description  \
0    1.0      Sarah  Sarah is a dedicated student with a passion fo...   
1    2.0       Alex  Alex is a tech enthusiast studying Computer Sc...   
2    3.0      Emily  Emily is an avid reader and historian, focusin...   
3    4.0      James  James is a science enthusiast excelling in Bio...   
4    5.0        Mia  Mia is a student of Economics and Political Sc...   
5    6.0       Ryan  Ryan is an aspiring engineer fascinated by Rob...   
6    7.0     Sophia  Sophia is a psychology enthusiast exploring th...   
7    8.0      Lucas  Lucas is an art aficionado studying Art Histor...   
8    9.0     Ethan   Ethan is a linguistics and anthropology studen...   
9   10.0     Olivia  Olivia is a budding environmental scientist sp...   
10  11.0       Lily  Lily is a dedicated student focusing on Chemis...   
11  12.0        Max  Max is an aspiring Computer Engineering studen...   
12  13.0        Ava  Ava delves into t

In [61]:
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk import pos_tag

# Read the CSV file into a pandas DataFrame
file_path = "/content/Final_pd.csv"
data = pd.read_csv(file_path)

# Function to extract tags from description
def extract_tags(description):
    # Tokenize the description
    tokens = word_tokenize(description)
    # Perform Part-of-Speech tagging
    tagged_words = pos_tag(tokens)
    # Define your logic to extract tags, for simplicity, let's assume the first noun encountered is the tag
    tags = [word for word, tag in tagged_words if tag.startswith('N')]
    return tags

# Create a dictionary to map names to tags
name_tag_dict = {}

# Iterate over each row to generate tags and map them to names
for index, row in data.iterrows():
    name = row['Name']
    description = row['Description']
    tags = extract_tags(description)
    name_tag_dict[name] = tags

# Display the dictionary
print(name_tag_dict)


{'Sarah': ['Sarah', 'student', 'passion', 'Mathematics', 'Physics', 'interests', 'realms', 'Astrophysics', 'Quantum', 'Mechanics', 'mysteries', 'universe', 'fervor', 'curiosity'], 'Alex': ['Alex', 'enthusiast', 'Computer', 'Science', 'Data', 'Science', 'interests', 'Artificial', 'Intelligence', 'Machine', 'Learning', 'cutting-edge', 'technologies', 'future'], 'Emily': ['reader', 'Literature', 'History', 'joy', 'Creative', 'Writing', 'narratives', 'Ancient', 'Civilizations', 'inspiration', 'past', 'endeavors'], 'James': ['James', 'science', 'enthusiast', 'excelling', 'Biology', 'Chemistry', 'interests', 'Genetics', 'Biochemistry', 'research', 'experiments', 'complexities', 'life', 'level'], 'Mia': ['Mia', 'student', 'Economics', 'Political', 'Science', 'passionate', 'dynamics', 'interests', 'International', 'Relations', 'Public', 'Policy', 'commitment', 'impact', 'society', 'decision-making'], 'Ryan': ['Ryan', 'engineer', 'Robotics', 'Renewable', 'Energy', 'interests', 'Systems', 'techn

In [64]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from sklearn.pipeline import Pipeline

# Read the CSV file into a pandas DataFrame
file_path = "/content/Final_pd.csv"
data = pd.read_csv(file_path)

# Function to extract tags from description
def extract_tags(description):
    # Tokenize the description
    tokens = word_tokenize(description)
    # Perform Part-of-Speech tagging
    tagged_words = pos_tag(tokens)
    # Define your logic to extract tags, for simplicity, let's assume the first noun encountered is the tag
    tags = [word for word, tag in tagged_words if tag.startswith('N')]
    return ' '.join(tags)

# Preprocess data
data['tags'] = data['Description'].apply(extract_tags)

# Split data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data['Description'], data['tags'], test_size=0.2, random_state=42)

# Create a pipeline with CountVectorizer and Multinomial Naive Bayes
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('clf', MultinomialNB())])

# Train the model
text_clf.fit(X_train, y_train)

# Test the model
predicted = text_clf.predict(X_test)

# Print accuracy
print("Accuracy:", text_clf.score(X_test, y_test))

# Example usage: Predict tags for a given phrase
def predict_tags(phrase):
    predicted_tags = text_clf.predict([phrase])
    return predicted_tags

# Example usage
phrase = "Student is very much interested in Mathematics and Physics"
predicted_tags = predict_tags(phrase)
print("Predicted tags for the phrase:", predicted_tags)


Accuracy: 0.0
Predicted tags for the phrase: ['Sarah student passion Mathematics Physics interests realms Astrophysics Quantum Mechanics mysteries universe fervor curiosity']
